In [1]:
from __future__ import division
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import copy
import random

N =  50                  # Number of nodes
rw = 700                 # Wealth value of rich
pw = 300                # Wealth value of poor
we_arr = [rw, pw]       # Array storing both the wealth values
pr = 0.5                # Probability of being rich
prob_arr = [pr, 1-pr]  # Array storing probability of wealth allotment values
M = 20                  # Number of rounds
co = 50                 # Unitary Cost of co-operation
be = 100                # Unitary Benefit of co-operation
re = 0.3                # Rewiring rate
S = 50                  # Number of iterations
pl = 0.3                # Initial probability of a link being formed between any two nodes
S1 = 8                  # Number of different lambda values

def inv_logit(y):
    inv_logit_out =(np.exp(y)/(1+np.exp(y)))
    return inv_logit_out

def gini_calc(x):       #IN: Wealth values across the population, OUT: Gini Coefficient
    sum2=0
    for i in range(len(x)):
        sum1=0
        for j in range(len(x)):
            sum1=sum1+np.abs(x[i]-x[j])
        sum2=sum2+sum1;
    
    gini=sum2/(2*len(x)**2*np.mean(x));
    return gini

node_arr = np.arange(N)
lam_arr = np.logspace(-6,0,num=S1,endpoint=True)
p0_arr = np.linspace(0.55,1.0,num = 6,endpoint=True)

result_array = np.zeros([S1,len(p0_arr),S])        # Storing final GINI coefficients over different rewiring rate and lambda values
result2_array = np.zeros([S1,len(p0_arr),S])       # Storing final cooperator fraction over different rewiring rate and lambda values
flag = 0

for h2 in range(len(p0_arr)):
    p0 = p0_arr[h2]
    for h1 in range(S1):    
        for h in range(S):             
            G = nx.erdos_renyi_graph(N, pl)
            A = nx.adjacency_matrix(G)
            E = G.number_of_edges()
            No_of_pairs = (N*(N-1))/2
            No_of_rewiring = round(No_of_pairs*re)
            Nre = int(No_of_rewiring)
            init_wealth_array = np.random.choice(we_arr,N,p=prob_arr)
                            
            GINI = gini_calc(init_wealth_array)                
            for k in range(M):                 # Continuing the game for M rounds
                if (k==0):                    # First round                 
                    decision_array = np.zeros(N)
                    for i in range(N):         # For all nodes in the network                        
                        a = np.random.random()                        
                        p_in = 0.7
                        if (a <= p_in):
                            decision_array[i] = 1       # Node-i decides to be cooperative                    
                            init_wealth_array[i] = init_wealth_array[i] - co*(G.degree(i))  # Updatation of wealth of a cooperative node
                            for l in range(N):          # Updatation of wealth of its neighbouring nodes
                                if (i==l):
                                    continue
                                else:
                                    if (A[i,l] == 1):
                                        init_wealth_array[l] = init_wealth_array[l] + be
                                    else:
                                        continue

                        else:
                            decision_array[i] = 0          # Node-i decides to be non-cooperative
                            continue               
                
                else:                     # Decision making in subsequent rounds
                    new_decision_array = np.zeros(N)
                    
                    for i in range(N):         # For all nodes in the network                                                            
                        cop = 0                     # Initializing the no of cooperator neighbours
                        for j in range(N):
                            if (A[i,j] == 1):               # If a link exists between nodes i and j
                                if (decision_array[j] == 1):        # If link j cooperated in last round
                                    cop += 1
                                elif (decision_array[j] == 0):      # If link j did not cooperate in last round
                                    continue
                            else:
                                continue
                
                        sum_neighbor_wealth = 0
                        for j in range(N):			# Total wealth of the neighbors
                            if (A[i,j] == 1):
                                sum_neighbor_wealth += init_wealth_array[j]
                            else:
                                continue
                 
                        if (G.degree(i)>0):
                            av_neighbor_wealth = (sum_neighbor_wealth)/(G.degree(i))
                            diff_wealth = av_neighbor_wealth - (init_wealth_array[i])
                            pc = p0 + (1-p0)*(np.tanh(lam_arr[h1]*diff_wealth)) - 0.10 
                        else:
                            pc = p0                   
                
                
                        if (G.degree(i)>0):              # For postive number of nearest neighbours
                            cop_rat = float(cop/G.degree(i))                    
                            if (cop_rat > 0.5):       # If cooperation parameter is greater than 0.5 i.e. cooperative environment
                                
                                c = np.random.random()
                                if (c < pc):
                                    new_decision_array[i] = 1       # Node-i decides to be cooperative                          
                                else:
                                    new_decision_array[i] = 0

                            elif (cop_rat < 0.5):          # Cooperation parameter is lesser than 0.5 i.e. selfish environment
                                
                                c = np.random.random()
                                if (c < pc):                        
                                    new_decision_array[i] = 0
                                else:
                                    new_decision_array[i] = 1
                            
                            elif (cop_rat == 0.5):     # Neutral environment
                                  
                                  c = np.random.random()
                                  if (c<=pc):
                                      new_decision_array[i] = 1
                                  else:
                                      new_decision_array[i] = 0
                                    
                        else:
                            new_decision_array[i] = 0
                            
                    for i in range(N):
                        if (new_decision_array[i] == 1): 
                            init_wealth_array[i] = init_wealth_array[i] - co*(G.degree(i))   # Updatation of wealth of a cooperative node
                            for l in range(N):          	# Updatation of wealth of its neighbouring nodes
                                if (i==l):
                                    continue
                                else:
                                    if (A[i,l] == 1):
                                        init_wealth_array[l] = init_wealth_array[l] + be
                                    else:
                                        continue
                        else:
                            continue                                  
                
                
                    decision_array = copy.copy(new_decision_array)   
                    
                if (k==M-1):
                    result_array[h1,h2,h] = gini_calc(init_wealth_array)
                    result2_array[h1,h2,h] = np.mean(new_decision_array)
                    
                # Rewiring step
                        
                temp_array = []                                                     
                                                                                      
                while (len(temp_array) < Nre) :            
                    i = random.choice(node_arr)
                    j = random.choice(node_arr)
                    
                    if (i==j):
                        continue
                    else:
                
                        if (i,j) in temp_array:
                            continue
                        elif (j,i) in temp_array:
                            continue
                        else:
                            temp_array.append((i,j))
                            temp = [i,j]
                                                             
                            if (A[i,j] == 1):                              # If a link exists between the selected pair
                                b = random.choice(temp)
                            
                                if (b == i):                               # If node-i is given the choice to break or not
                                    if (decision_array[j] == 0):
                                        c = np.random.random()
                                        if (c < 0.7):
                                            G.remove_edge(i,j)                 # Breakage of link for node-i disagreeing
                                  
                                        else:
                                            continue                        # Link is kept intact
                                    else:
                                        c = np.random.random()
                                        if (c < 0.87):
                                            continue
                                        else:
                                            G.remove_edge(i,j)
                                else:                                   # If node-j is given the choice to break or not	
                                    if (decision_array[i] == 0):
                                        c = np.random.random()
                                        if (c < 0.7):                                    
                                            G.remove_edge(i,j)         # Breakage of link for node-j disagreeing
                                        else:
                                            continue			# Link is kept intact
                                    else:
                                        c = np.random.random()
                                        if (c < 0.87):
                                            continue
                                        else:
                                            G.remove_edge(i,j)
                                                                
            
                            else:                 # If a link does not exist between the selected pair
                                if (decision_array[i] == 1 and decision_array[j] == 1):
                                    c = np.random.random()
                                    if (c < 0.93):
                                        G.add_edge(i,j)
                                    else:
                                        continue
                            
                                if (decision_array[i] == 0 and decision_array[j] == 0):
                                    c = np.random.random()
                                    if (c < 0.80):
                                        continue
                                    else:
                                        G.add_edge(i,j)
                        
                                if (decision_array[i] == 0 and decision_array[j] == 1):
                                    c = np.random.random()
                                    if (c < 0.70):
                                        continue
                                    else:
                                        G.add_edge(i,j)
                                
                                if (decision_array[i] == 1 and decision_array[j] == 0):
                                    c = np.random.random()
                                    if (c < 0.70):
                                        continue
                                    else:
                                        G.add_edge(i,j)
                                
                A = nx.adjacency_matrix(G)    # Adjacency matrix of the network            
                               
        flag += 1
        yo = (flag*100)/(S1*len(p0_arr))        
                
        print('Job completion percentage: %f' %yo)

np.savez('lam_vs_p0', result_array=result_array, result2_array=result2_array)    



KeyboardInterrupt: 